In [1]:
!curl http://localhost:9200/

{
  "name" : "b52d89a397f3",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "BXOEw3B2RvqDXI3j7fObwQ",
  "version" : {
    "number" : "8.3.1",
    "build_type" : "docker",
    "build_hash" : "b9a6b2867996ba92ceac66cb5bafc6db25e7910e",
    "build_date" : "2022-06-29T18:39:55.731992798Z",
    "build_snapshot" : false,
    "lucene_version" : "9.2.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [2]:
import json

from elasticsearch import Elasticsearch, helpers
from collections import deque

In [3]:
def data_gen(): 
    with open('data_hub/data.json', 'r') as f:
        documents = json.load(f)  
           
    for doc in documents:
        json_data = {}
        json_data['article_title'] = doc['article_title']
        json_data['section_title'] = doc['section_title']
        json_data['main_section'] = doc['main_section']
        json_data['section_number'] = doc['section_number']
        json_data['page_id'] = doc['page_id']
        json_data['text'] = doc['text']
        json_data['source_url'] = doc['source_url']
        json_data['tags'] = doc['tags']
        yield json_data 

In [4]:
client = Elasticsearch("http://localhost:9200")

# Bulk load data into Elasticsearch
client.indices.create(index='disease_docs')
deque(helpers.parallel_bulk(client, data_gen(), index='disease_docs'), maxlen=0)
client.indices.refresh()

ObjectApiResponse({'_shards': {'total': 4, 'successful': 2, 'failed': 0}})

In [10]:
# Search documents in Elasticsearch
query = {'query':{'match_phrase':{'text':'Black Death'}}}

hits = client.search(index='disease_docs', body=query) 
hits['hits']

/tmp/ipykernel_78147/2664752975.py:4: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  hits = client.search(index='disease_docs', body=query)


{'total': {'value': 3, 'relation': 'eq'},
 'max_score': 6.973024,
 'hits': [{'_index': 'disease_docs',
   '_id': 'nFUnx4EBf2mbjDV_iX-w',
   '_score': 6.973024,
   '_ignored': ['text.keyword'],
   '_source': {'article_title': 'Pandemic',
    'section_title': 'Summary',
    'main_section': 'Summary',
    'section_number': 0,
    'page_id': 24255,
    'text': 'A pandemic (from Greek πᾶν, pan, "all" and δῆμος, demos, "people" the \'crowd\') is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people. A widespread endemic disease with a stable number of infected people is not a pandemic. Widespread endemic diseases with a stable number of infected people such as recurrences of seasonal influenza are generally excluded as they occur simultaneously in large regions of the globe rather than being spread worldwide.\nThroughout human history, there have been a number of pandemics of disease

In [9]:
client.cat.count(index='_all', format='json')

ListApiResponse([{'epoch': '1656903820', 'timestamp': '03:03:40', 'count': '393'}])